<a href="https://colab.research.google.com/github/Valeriy1990/Neural_networks/blob/main/%D0%A2%D0%B5%D0%BB%D0%B5%D0%B3%D1%80%D0%B0%D0%BC-%D0%B1%D0%BE%D1%82%2C%20%D0%BE%D1%82%D0%B2%D0%B5%D1%87%D0%B0%D1%8E%D1%89%D0%B8%D0%B9%20%D0%BD%D0%B0%20%D0%B2%D0%BE%D0%BF%D1%80%D0%BE%D1%81%D1%8B%20%D0%B8%D0%B7%20%D0%B1%D0%B0%D0%B7%D1%8B%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%20(%D0%BF%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3%20%D0%92%D0%B8%D0%BA%D0%B8%D0%BF%D0%B5%D0%B4%D0%B8%D0%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Телеграм-бот, который будет отвечать на вопросы из вашей базы знаний.




In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
!pip install wikipedia-api aiogram asyncio nest_asyncio openai tiktoken

In [15]:
import nest_asyncio
import asyncio
import logging
from aiogram import Bot, Dispatcher, types
from aiogram.filters import Command
import tiktoken
import os
import getpass
from openai import OpenAI
from google.colab import userdata
from scipy import spatial  # вычисляет сходство векторов
import pandas as pd
import ast
import re  # для вырезания ссылок <ref> из статей Википедии
import time
from google.colab import files
import random
import wikipediaapi
import nltk
from nltk.tokenize import sent_tokenize
import json
import numpy as np
from numpy.linalg import norm

nest_asyncio.apply()
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
# Использование ключа API от VseGpt
key = userdata.get('VSEGPT_API_KEY')
os.environ["OPENAI_API_KEY"] = key

# Адрес сервера VseGpt
base_url = "https://api.vsegpt.ru/v1"
os.environ["OPENAI_BASE_URL"] = base_url

# Выбираем модель GPT
# GPT_MODEL = "gpt-3.5-turbo"


client = OpenAI(
  api_key=key,
  base_url="https://api.vsegpt.ru/v1"
)




1. Подготовьте данные для поиска

In [17]:
# Замените 'ВАША_МОДЕЛЬ_ДЛЯ_ТОКЕНИЗАЦИИ' на имя модели, соответствующей вашей модели OpenAI
# Например, для моделей gpt-3.5-turbo или gpt-4 используйте "cl100k_base"
encoding = tiktoken.get_encoding("cl100k_base")

def get_embedding(text, model="text-embedding-3-small"):
    try:
        text = text.replace("\n", " ")
        response = client.embeddings.create(
            input=[text],
            model=model
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Ошибка при получении эмбеддинга: {e}")
        return None

def count_tokens(text: str) -> int:
    """Считает количество токенов в тексте с использованием токенайзера."""
    if not text:
        return 0
    return len(encoding.encode(text))

In [18]:
# Инициализация Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia(
    language='en',
    extract_format=wikipediaapi.ExtractFormat.WIKI,
    user_agent='MyWikipediaBot/1.0'
)

# Конфигурация
CATEGORY_NAME = "Category:Psychology"
IGNORED_SECTIONS = [
    "See also",
    "References",
    "External links",
    "Further reading",
    "Footnotes",
    "Bibliography",
    "Sources",
    "Citations",
    "Literature",
    "Footnotes",
    "Notes and references",
    "Photo gallery",
    "Works cited",
    "Photos",
    "Gallery",
    "Notes",
    "References and sources",
    "References and notes",
]
MAX_TOKENS = 4000

In [19]:
def split_into_chunks(text, max_tokens=MAX_TOKENS):
    def count_tokens(text):
        return len(text) // 4

    if count_tokens(text) <= max_tokens:
        return [text]

    paragraphs = text.split('\n\n')
    if len(paragraphs) > 1:
        chunks = []
        for paragraph in paragraphs:
            if paragraph.strip():
                chunks.extend(split_into_chunks(paragraph, max_tokens))
        return chunks

    sentences = sent_tokenize(text, language='english')
    if len(sentences) > 1:
        chunks = []
        current_chunk = []
        current_token_count = 0

        for sentence in sentences:
            sentence_tokens = count_tokens(sentence)

            if current_token_count + sentence_tokens > max_tokens and current_chunk:
                chunks.append(' '.join(current_chunk))
                current_chunk = [sentence]
                current_token_count = sentence_tokens
            else:
                current_chunk.append(sentence)
                current_token_count += sentence_tokens

        if current_chunk:
            chunks.append(' '.join(current_chunk))

        return chunks

    mid = len(text) // 2
    split_pos = text.rfind(' ', 0, mid) or text.find(' ', mid)
    if split_pos == -1:
        split_pos = mid

    part1 = text[:split_pos]
    part2 = text[split_pos:]

    return split_into_chunks(part1, max_tokens) + split_into_chunks(part2, max_tokens)

def clean_text(text):
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def process_article(title):
    page = wiki_wiki.page(title)
    if not page.exists():
        return []

    sections = []
    for section in page.sections:
        if section.title in IGNORED_SECTIONS:
            continue

        section_text = f"Заголовок: {section.title}\n{clean_text(section.text)}"
        section_chunks = split_into_chunks(section_text)
        sections.extend(section_chunks)

    return sections

def get_category_members(category, depth=1):
    if depth == 0:
        return []

    pages = []
    for member in category.categorymembers.values():
        if member.ns == wikipediaapi.Namespace.CATEGORY and depth > 1:
            pages.extend(get_category_members(member, depth-1))
        elif member.ns == wikipediaapi.Namespace.MAIN:
            pages.append(member)

    return pages

In [20]:
# Сбор данных
print("Сбор данных из Википедии...")
category = wiki_wiki.page(CATEGORY_NAME)
if not category.exists():
    print(f"Категория {CATEGORY_NAME} не найдена!")
else:
    articles = get_category_members(category)
    knowledge_base = []

    for article in articles:
        print(f"Обработка статьи: {article.title}")
        fragments = process_article(article.title)  # список строк

        for fragment in fragments:
            try:
                embedding = get_embedding(fragment)
                knowledge_base.append({
                    "text": fragment,
                    "embedding": embedding,
                    "source": article.title  # опционально: откуда фрагмент
                })
                print(f"  ✅ Эмбеддинг для фрагмента создан")
            except Exception as e:
                print(f"  ❌ Ошибка при создании эмбеддинга: {e}")

    print(f"База знаний содержит {len(knowledge_base)} фрагментов с эмбеддингами")

Сбор данных из Википедии...
Обработка статьи: Psychology
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
Обработка статьи: Outline of psychology
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
Обработка статьи: Alarmism
  ✅ Эмбеддинг для фрагмента создан
Обработка статьи: Aphantasia
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эмбеддинг для фрагмента создан
Обработка статьи: Belief congruence
  ✅ Эмбеддинг для фрагмента создан
  ✅ Эм

In [21]:
# Сохранение
with open("knowledge_base.json", "w", encoding="utf-8") as f:
    json.dump(knowledge_base, f, ensure_ascii=False, indent=2)

print("База знаний сохранена в knowledge_base.json")

База знаний сохранена в knowledge_base.json


In [22]:
def cosine_similarity(a, b):
    """Вычисляет косинусное сходство между двумя векторами."""
    return np.dot(a, b) / (norm(a) * norm(b))

In [23]:
# Создание Telegram-бота
API_TOKEN = userdata.get('API_TOKEN')
bot = Bot(token=API_TOKEN)
dp = Dispatcher()

# Команда /help
@dp.message(Command("help"))
async def send_help(message: types.Message):
    help_text = (
        "Тематика: Научные статьи из Википедии\n"
        f"Число записей: {len(knowledge_base)}\n"
        "Пример запроса: 'What is Alarmism?'"
    )
    await message.answer(help_text)

@dp.message()
async def handle_query(message: types.Message):
    query = message.text
    user_id = message.from_user.id

    # 1. Считаем токены запроса
    query_tokens = count_tokens(query)
    if query_tokens > 4000:
        await message.answer("Ваш запрос слишком длинный (более 4000 токенов).")
        return

    try:
        # 2. Получаем эмбеддинг запроса
        query_embedding = get_embedding(query)
    except Exception as e:
        print(f"Ошибка при получении эмбеддинга запроса: {e}")
        await message.answer("Не удалось обработать ваш запрос.")
        return

    # 3. Считаем сходство для всех фрагментов
    scored_chunks = []
    for chunk in knowledge_base:
        similarity = cosine_similarity(query_embedding, chunk["embedding"])
        text_tokens = count_tokens(chunk["text"])
        scored_chunks.append({
            "text": chunk["text"],
            "source": chunk.get("source", "Неизвестно"),
            "similarity": similarity,
            "tokens": text_tokens
        })

    # 4. Сортируем по релевантности (от самого похожего к самому непохожему)
    scored_chunks.sort(key=lambda x: x["similarity"], reverse=True)

    # 5. Набираем контекст, пока не превысим 4000 токенов
    relevant_chunks = []
    current_tokens = query_tokens  # начинаем с токенов вопроса

    # Добавим токены системного промпта — это важно!
    system_prompt = "You are a helpful assistant that answers questions based on the provided context."
    system_tokens = count_tokens(system_prompt)
    current_tokens += system_tokens

    # Также учтём токены на оформление контекста (заголовки, разделители и т.п.)
    # Примерно: "Ответь на вопрос... \n\nВопрос: ..." — добавим запас 50 токенов
    overhead_tokens = 50
    current_tokens += overhead_tokens

    # Теперь добавляем фрагменты, пока не превысим лимит
    for chunk in scored_chunks:
        if current_tokens + chunk["tokens"] <= 4000:
            relevant_chunks.append(chunk)
            current_tokens += chunk["tokens"]
        else:
            break  # Прекращаем, если следующий фрагмент выведет за лимит

    if not relevant_chunks:
        await message.answer("Не удалось найти подходящую информацию в пределах лимита токенов.")
        return

    # 6. Формируем контекст
    context = (
        "Ответь на следующий вопрос, используя предоставленную информацию. "
        "Если информация отсутствует, скажи, что не можешь ответить.\n\n"
    )
    for chunk in relevant_chunks:
        context += chunk["text"] + "\n\n"
    context += f"Вопрос пользователя: {query}"

    # 7. Отправляем запрос в модель
    try:
        response = client.chat.completions.create(
            # model="deepseek-chat-v3-0324:free",
            model="openai/gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": context}
            ]
        )
        answer = response.choices[0].message.content

        # Опционально: добавляем источники
        sources = set(chunk["source"] for chunk in relevant_chunks)
        if sources:
            answer += "\n\n📚 Источники: " + ", ".join(sources)

        await message.answer(answer)

    except Exception as e:
        print(f"Ошибка при обращении к API: {e}")
        await message.answer("Произошла ошибка при генерации ответа.")


In [24]:
# Функция для запуска бота
async def main():
    print("Бот запущен")
    await dp.start_polling(bot)

# Запуск бота
print("Запуск бота...")
try:
    asyncio.run(main())
except Exception as e:
    print(f"Ошибка при запуске бота: {e}")

Запуск бота...
Бот запущен
